In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import joblib
import glob

from data_utils.mg_sg_generator import get_dataset_ids, get_dataset_for, prepare_dataset_for_training
from data_utils.mg_sg_generator import MotiongramSpectrogramGenerator

import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

In [ ]:
fout = get_dataset_for(nfft=256)
for k, v in fout.items():
    print(f'[{k}] Wrote {v[0]} files to {v[1]}')

In [ ]:
def parse_tfr_example(example):
    data = {
        'labels'      : tf.io.FixedLenFeature([], tf.string),
        'memlen'      : tf.io.FixedLenFeature([],  tf.int64),
        'motiongram'  : tf.io.FixedLenFeature([], tf.string),
        'spectrogram' : tf.io.FixedLenFeature([], tf.string)
    }
    content = tf.io.parse_single_example(example, data)
    mg, sg = content['motiongram'], content['spectrogram']
    shapelen = content['memlen']
    mg_feature = tf.reshape(tf.io.parse_tensor(mg, out_type=tf.float32), shape=[shapelen])
    sg_feature = tf.reshape(tf.io.parse_tensor(sg, out_type=tf.float32), shape=[shapelen])
    return mg_feature, sg_feature

def get_dataset_small(filename):
    # create the dataset
    dataset = tf.data.TFRecordDataset(filename)

    # pass every single feature through our mapping function
    dataset = dataset.map(
      parse_tfr_example
    )
    
    return dataset

def input_fn(filename, batch_size):
    ds = get_dataset_small(filename)
    ds = ds.shuffle(10000)
    ds = ds.batch(batch_size)
    ds = ds.repeat()
    #ds = ds.prefetch(tf.data.AUTOTUNE)
    return ds

### Setup a simple test model 

In [ ]:
inp = Input(shape=(129*128, ))
x = Dense(100)(inp)
out = Dense(129*128)(x)

model = Model(inputs=[inp], outputs=[out])
model.compile(optimizer="adam", loss="mse", metrics=["accuracy"])

In [ ]:
# Set batch size
batch_size = 10

ds_train      = prepare_dataset_for_training(fout["train"][-1], batch_size)
#input_fn(fout["train"][-1], batch_size=batch_size)
ds_validation = prepare_dataset_for_training(fout["validation"][-1], batch_size)
#input_fn(fout["validation"][-1], batch_size=batch_size)

In [ ]:
model.fit(
    ds_train,
    steps_per_epoch=1056//batch_size,
    validation_data=ds_validation,
    validation_steps=227//batch_size,
    epochs=20
)

In [ ]:
import numpy as np
iterator = iter(ds_train)
tf_mg, tf_sg = iterator.get_next()
mg, sg = tf_mg.numpy(), tf_sg.numpy()

mg_0 = np.reshape(mg[0, :], newshape=(129, 128))
sg_0 = np.reshape(sg[0, :], newshape=(129, 128))
#mg_1 = np.reshape(mg[-1, :], newshape=(129, 128))
#sg_1 = np.reshape(sg[-1, :], newshape=(129, 128))

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(10, 10))
axs[0, 0].set_xticks([])
axs[0, 0].set_yticks([])
axs[0, 0].set_xlabel("(a) Input Motiongram")
axs[0, 0].imshow(mg_0, aspect="auto", cmap="Spectral_r", interpolation="bicubic")

#axs[0, 1].set_xticks([])
#axs[0, 1].set_yticks([])
#axs[0, 1].set_xlabel("(a) Input Motiongram")
#axs[0, 1].imshow(mg_1, aspect="auto", cmap="Spectral_r", interpolation="bicubic")

axs[1, 0].set_xticks([])
axs[1, 0].set_yticks([])
axs[1, 0].set_xlabel("(a) Input Motiongram")
axs[1, 0].imshow(np.flipud(sg_0), aspect="auto", cmap="Spectral_r", interpolation="bicubic")

#axs[1, 1].set_xticks([])
#axs[1, 1].set_yticks([])
#axs[1, 1].set_xlabel("(a) Input Motiongram")
#axs[1, 1].imshow(np.flipud(sg_1), aspect="auto", cmap="Spectral_r", interpolation="bicubic")

plt.show()